<a href="https://colab.research.google.com/github/sonamoni927/COUSTOMER-SALES-ANALYSIS/blob/main/_Coustomer_salesAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv("/content/Sales Dataset.csv")

print("==== RAW DATA PREVIEW ====")
print(df.head())


==== RAW DATA PREVIEW ====
  Order ID  Amount  Profit  Quantity     Category      Sub-Category  \
0  B-26776    9726    1275         5  Electronics  Electronic Games   
1  B-26776    9726    1275         5  Electronics  Electronic Games   
2  B-26776    9726    1275         5  Electronics  Electronic Games   
3  B-26776    4975    1330        14  Electronics          Printers   
4  B-26776    4975    1330        14  Electronics          Printers   

  PaymentMode  Order Date   CustomerName     State     City Year-Month  
0         UPI  2023-06-27  David Padilla   Florida    Miami    2023-06  
1         UPI  2024-12-27  Connor Morgan  Illinois  Chicago    2024-12  
2         UPI  2021-07-25   Robert Stone  New York  Buffalo    2021-07  
3         UPI  2023-06-27  David Padilla   Florida    Miami    2023-06  
4         UPI  2024-12-27  Connor Morgan  Illinois  Chicago    2024-12  


In [ ]:

df = df.drop_duplicates()



In [ ]:
df['Quantity'] = df['Quantity'].fillna(0)
# Calculate 'Unit_Price'. To avoid division by zero errors and infinite values,
# we'll set 'Unit_Price' to NaN where 'Quantity' is 0, otherwise perform the division.
df['Unit_Price'] = df.apply(lambda row: row['Amount'] / row['Quantity'] if row['Quantity'] != 0 else pd.NA, axis=1)
df['Unit_Price'] = df['Unit_Price'].fillna(df['Unit_Price'].mean())

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

In [ ]:

df = df.dropna(subset=['Order Date'])


In [ ]:


# Create a new column
df['Total_Sales'] = df['Unit_Price'] * df['Quantity']

# Save clean data
df.to_csv("clean_sales_data.csv", index=False)

print("Cleaning Completed. File saved as clean_sales_data.csv")


Cleaning Completed. File saved as clean_sales_data.csv


In [ ]:
import sqlite3

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Execute the SQL statement to create the table
conn.execute("""
CREATE TABLE sales_cleaned (
    order_id INT,
    product VARCHAR(100),
    category VARCHAR(50),
    order_date DATE,
    unit_price FLOAT,
    quantity INT,
    total_sales FLOAT
);
""")

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Table 'sales_cleaned' successfully created in an in-memory SQLite database.")

Table 'sales_cleaned' successfully created in an in-memory SQLite database.


In [ ]:
import pandas as pd
import sqlite3

df = pd.read_csv("/content/clean_sales_data.csv")

conn = sqlite3.connect("sales.db")
df.to_sql("sales_cleaned", conn, if_exists="replace", index=False)

print("Data loaded into SQL database successfully.")


Data loaded into SQL database successfully.


In [ ]:
import sqlite3
import pandas as pd

# Connect to the sales database
conn = sqlite3.connect('sales.db')

# Execute the SQL query using pandas to get the result as a DataFrame
null_counts_df = pd.read_sql_query("""
SELECT
    SUM(CASE WHEN "Sub-Category" IS NULL THEN 1 ELSE 0 END) AS null_sub_category,
    SUM(CASE WHEN category IS NULL THEN 1 ELSE 0 END) AS null_category
FROM sales_cleaned;
""", conn)

# Print the results
print(null_counts_df)

# Close the connection
conn.close()

   null_sub_category  null_category
0                  0              0


In [ ]:
import sqlite3

# Connect to the sales database
conn = sqlite3.connect('sales.db')

# Execute the SQL DELETE statement
conn.execute("""
DELETE FROM sales_cleaned
WHERE quantity <= 0;
""")

# Commit the changes to the database
conn.commit()

# Close the connection
conn.close()

print("Rows with quantity less than or equal to 0 deleted from sales_cleaned.")

Rows with quantity less than or equal to 0 deleted from sales_cleaned.


In [ ]:
import sqlite3

# Connect to the sales database
conn = sqlite3.connect('sales.db')
cursor = conn.cursor()

# Function to check if a column exists
def column_exists(cursor, table_name, column_name):
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = [col[1] for col in cursor.fetchall()]
    return column_name in columns

# Add the 'month' column if it doesn't exist
if not column_exists(cursor, 'sales_cleaned', 'month'):
    conn.execute("ALTER TABLE sales_cleaned ADD COLUMN month INT;")
    print("Column 'month' added.")
else:
    print("Column 'month' already exists, skipping addition.")

# Add the 'year' column if it doesn't exist
if not column_exists(cursor, 'sales_cleaned', 'year'):
    conn.execute("ALTER TABLE sales_cleaned ADD COLUMN year INT;")
    print("Column 'year' added.")
else:
    print("Column 'year' already exists, skipping addition.")

# Update the 'month' and 'year' columns
conn.execute("""
UPDATE sales_cleaned
SET month = STRFTIME('%m', "Order Date"),
    year = STRFTIME('%Y', "Order Date");
""")

# Commit the changes
conn.commit()

# Close the connection
conn.close()

print("Columns 'month' and 'year' populated successfully.")

Column 'month' already exists, skipping addition.
Column 'year' already exists, skipping addition.
Columns 'month' and 'year' populated successfully.


In [ ]:
import sqlite3
import pandas as pd

# Connect to the sales database
conn = sqlite3.connect('sales.db')

# Execute the SQL query using pandas to get the result as a DataFrame
top_products_df = pd.read_sql_query("""
SELECT "Sub-Category", SUM(total_sales) AS revenue
FROM sales_cleaned
GROUP BY "Sub-Category"
ORDER BY revenue DESC
LIMIT 10;
""", conn)

# Print the results
print(top_products_df)

# Close the connection
conn.close()

       Sub-Category   revenue
0           Markers  627875.0
1            Tables  625177.0
2             Sofas  568367.0
3          Printers  566359.0
4  Electronic Games  565092.0
5              Pens  552269.0
6             Paper  524755.0
7            Phones  503055.0
8            Chairs  431964.0
9           Laptops  419950.0


In [ ]:
import sqlite3
import pandas as pd

# Connect to the sales database
conn = sqlite3.connect('sales.db')

# Execute the SQL query using pandas to get the result as a DataFrame
category_revenue_df = pd.read_sql_query("""
SELECT category, SUM(total_sales) AS revenue
FROM sales_cleaned
GROUP BY category
ORDER BY revenue DESC;
""", conn)

# Print the results
print(category_revenue_df)

# Close the connection
conn.close()

          Category    revenue
0  Office Supplies  2089510.0
1      Electronics  2054456.0
2        Furniture  2038673.0


In [ ]:
import sqlite3
import pandas as pd

# Connect to the sales database
conn = sqlite3.connect('sales.db')

# Execute the SQL query using pandas to get the result as a DataFrame
monthly_revenue_df = pd.read_sql_query("""
SELECT year, month, SUM(total_sales) AS revenue
FROM sales_cleaned
GROUP BY year, month
ORDER BY year, month;
""", conn)

# Print the results
print(monthly_revenue_df)

# Close the connection
conn.close()

    year  month   revenue
0   2020      3   22991.0
1   2020      4  133385.0
2   2020      5  113287.0
3   2020      6   46900.0
4   2020      7   38556.0
..   ...    ...       ...
56  2024     11   60976.0
57  2024     12   98879.0
58  2025      1  112906.0
59  2025      2   84712.0
60  2025      3   52198.0

[61 rows x 3 columns]
